<a href="https://colab.research.google.com/github/yhkondo/laj-ML/blob/main/lajml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#　Googleドライブをマウントしておく。
# 次のサイトを参照。　https://www.lajdb.org/TOP.html　の「経緯度情報付きエクセルファイル（形式２）」を利用
# かたつむり
!wget -P '/content/drive/MyDrive/' https://www.lajdb.org/lajdb_data/data02_xls/Q005_excelDataWithCordinateData_TKY_JGD_01.xls
# くるぶし
!wget -P '/content/drive/MyDrive/' https://www.lajdb.org/lajdb_data/data02_xls/Q233_excelDataWithCordinateData_TKY_JGD_01.xls

In [ ]:
#　pandasからエクセルのxlsファイルを読む時にxlrdのバージョンあわせが必要になる
!pip install xlrd==1.2.0

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import folium
#import matplotlib.pyplot as plt


#　下のかたつむり、くるぶし、いずれかのファイルを使う。もちろん、他のものでもよい。

# 日本言語地図の部分図（かたつむり）
input_file_name = '/content/drive/MyDrive/Q005_excelDataWithCordinateData_TKY_JGD_01.xls'

# くるぶし　
#input_file_name = '/content/drive/MyDrive/Q233_excelDataWithCordinateData_TKY_JGD_01.xls'


# 地点アイコン抜き出し率(２８００地点のうち、n分の1になる。1にすると全部出るが重い。）
n = 3

# 語形ベクトルの次元数（アルファベットの数）
number_of_alphabets = 26

# 語形ベクトルの圧縮後の次元数(2が適切）
dimention = 2

# 地図の初期拡大割合
zoom_init = 6
# 地図の初期センター緯度経度（東京駅）
lat_init = 35.681167
lon_init = 139.767052

# アイコンの形
icon_shape = 'home'
#　アイコンの色数(6か7か8辺りが適切か。）
color_numbers = 7
# アイコンの色リスト、とりあえず9まで用意。全部使わなくともよい。
color_list = ['lightgreen', 'red', 'lightblue', 'gray', 'darkgreen', 'darkblue', 'pink', 'purple', 'orange']



# UTF-8ユニコード英字大文字を整数（0から26）にする関数

def chr_to_int(character):
    chr_code = ord(character)
    chr_a_code = ord('A') 
    return int(chr_code - chr_a_code)


#語形をベクトル化(Bag of Characters）する関数

def word_to_vector(word):
#　ベクトルの０による初期化(半角英字大文字数）
    bag_of_chr_vector = np.zeros(number_of_alphabets, dtype = int)
#語形を字に分解してベクトルに頻度表を作る
# bag of character unigramということになる。 bigramにするなどの方法もあるだろう
    for character_in_word in word:
# 語形欄の半角英字以外の記号は除外。二重語形やNRもそのまま算入。
        if character_in_word.isalpha() and character_in_word.isascii():
            bag_of_chr_vector[chr_to_int(character_in_word)] += 1
    return bag_of_chr_vector



#　pandasのデータフレームに元エクセルシートを読み込み
original_book = pd.ExcelFile(input_file_name)
original_df = original_book.parse()

#必要な列だけを取り出し、さらに地図用の新列名を付加する
df = original_df.loc[:, ['no', 'x0IJGD', 'y0IJGD', '語形']]
df['アイコン色'] = ''


#　語形のベクトル化
i = 0
word = df.iloc[i]['語形']
all_vector = word_to_vector(word)
for i in range(1,len(df)):
    word = df.iloc[i]['語形']
    all_vector = np.vstack([all_vector,  word_to_vector(word)])


#　語形ベクトルの低次元への圧縮
model1 = PCA(n_components = dimention)
model1.fit(all_vector)
model1_trans = model1.transform(all_vector)

#print('主成分の分散割合:{}'.format(model1.explained_variance_ratio_))
#print('model1 shape:{}'.format(model1_trans.shape))

#　低次元に圧縮したものをクラスタに分類
model2 = KMeans(n_clusters = color_numbers)
model2.fit(model1_trans)
num_labels = len(model2.labels_)

# クラスタを散布図に示す（テスト用）
#fig=plt.figure(figsize=(20,12),facecolor='w')
#for i in range(num_labels):
#    plt.scatter(model1_trans[i, 0], model1_trans[i, 1], color=color_list[model2.labels_[i]])
#plt.show()

# クラスタ番号から得たアイコン色をｄｆに書き込み
color_array = []
for i in range(num_labels):
   color_array.append(color_list[model2.labels_[i]])
df['アイコン色'] = color_array

#　抜き出す行だけを使って縮小版ｄｆにする（ｎ分の1）
reduced_df = df.query('index % @n == 0')

#地図作り
#地図の初期化
map = folium.Map(

#初期地図センター位置
   location = [lat_init, lon_init],
#初期拡大値
   zoom_start = zoom_init,
#地図のスタイル
   tiles = "OpenStreetMap"

)


# 地図地点アイコン追加

#　縮小したdfを利用
for i in range(len(reduced_df)):
   folium.Marker(

#　緯度経度
        location = [reduced_df.iloc[i]['y0IJGD'], reduced_df.iloc[i]['x0IJGD']],
#　ポップアップする語形
        popup = str(reduced_df.iloc[i]['語形']),
#　アイコン色
        icon = folium.Icon(color=reduced_df.iloc[i]['アイコン色'], icon=icon_shape)
   ).add_to(map)


#マップの描画
map
# マップの保存（カレントディレクトリへ）
#map.save('map.html')